In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [2]:
#Add functions for Univariate analysis
def describe_numerical_col(df, col_name):
    info = df[[col_name]].describe().to_dict()[col_name]
    info['shapiro'] = f'{stats.shapiro(df[col_name])[1]: .5f}'
    info['normal'] = float(info['shapiro']) < 0.05
    info['missing'] = df[col_name].isna().sum()
    info['skew'] = f'{stats.skew(df[col_name]):.5f}'
    info['type'] = ('slight ' if info['normal'] else '') +\
        ('right(positive)' if float(info['skew']) > 0 else 'left(negative)') + '-skew'

    fig, ax = plt.subplots(2, 2, figsize=(16, 7), gridspec_kw={'height_ratios':(.85, .15)})
    sns.histplot(df[col_name], kde=True, ax=ax[0, 0], color='#55A868')
    sns.boxplot(df[col_name], orient='h', ax=ax[1, 0], color="#5583A8")
    counts, bin_edges = np.histogram(df[col_name], bins=10, density = True)
    pdf = counts / (sum(counts))
    cdf = np.cumsum(pdf)
    ax[1, 1] = plt.subplot(122)
    plt.plot(bin_edges[1:], pdf, label='PDF')
    plt.plot(bin_edges[1:], cdf, label='CDF')
    plt.legend()
    ax[0, 0].set_xticklabels([])
    ax[1, 0].set_yticklabels([])
    ax[0, 0].set_xlabel('')
    ax[0, 0].set_ylabel('Count')
    fig.suptitle(col_name, fontsize=30)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    info_df = pd.DataFrame.from_dict(info, orient='index', columns=[''])
    print('='*18 + ' ' +col_name + ' ' + '='*18)
    print(info_df)
    print('='*40)
    
def categorize_numerical_col(col_name, bins, bins_name):
    new_col = f'{col_name}_categorized'
    df[new_col] = pd.cut(df[col_name], bins=bins, labels=bins_name)

def describe_categorical_col(df, col_name):
    counts = pd.DataFrame(df[col_name].value_counts()).reset_index()
    counts.columns = ['Group', 'Count']
    total = sum(counts['Count'])
    counts['%'] = (counts['Count'] / total * 100).round(2)

    fig = px.pie(
        counts,
        names='Group',
        values='Count',
        title=f'<b>Distribution of {col_name} in each group</b>',
        color='Group',
        hole=0.3
    )

    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',

    )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Groups',
        font=dict(family="Arial, sans-serif", size=14)
    )
    fig.show()
    print('='*18 + ' ' +col_name + ' ' + '='*18)
    print(counts)
    print('='*40)
    # plt.figure(figsize=(12, 7))
    # ax = sns.barplot(
    #     data=counts,
    #     x=counts[col_name],
    #     y=counts['count'],
    #     palette='viridis'
    # )

    # for i in range(len(ax.patches)):
    #     p = ax.patches[i]
    #     percentage = counts['percents'][i]
    #     ax.annotate(f'{percentage:.2f}%',
    #                 (p.get_x() + p.get_width()/2., p.get_height()),
    #                 ha='center', va='center',
    #                 fontsize=11, color='black',
    #                 xytext=(0, 10),
    #                 textcoords='offset points')
    
    # plt.ylim(0, max(counts['count']) * 1.15)
    # ax.set_xlabel('')
    # ax.set_title(f'Distribution of {col_name} in each group')
    # plt.show()


In [64]:
df = pd.read_csv('marketing.csv')

In [65]:
df.head()

,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
0,13300,36,Female,47648.0,Email,Consideration,893.7554921892273,0.132078,0.198063,38,6.900081,11.923316,9,3,6,1,1139,IsConfid,ToolConfid,1
1,10429,32,Female,47491.0,Email,Consideration,7330.251375088103,0.120810,0.088416,27,4.589390,NaN,33,0,2,4,3772,IsConfid,ToolConfid,1
2,10557,52,Male,83830.0,Referral,Awareness,7720.360685483039,0.125472,0.082197,35,7.403157,0.660581,29,1,3,0,3611,IsConfid,ToolConfid,1
3,12602,52,Male,116342.0,Social Media,Consideration,8337.45315727269,0.158739,0.089032,25,9.367090,12.218421,72,3,7,4,1173,IsConfid,ToolConfid,1
4,8592,58,Female,90994.0,Email,Retention,5273.098832292671,0.226054,0.099792,13,7.968767,4.894079,41,16,7,0,1102,IsConfid,ToolConfid,1


In [66]:
df.isna().sum()

CustomerID               0
Age                      0
Gender                   0
Income                  79
CampaignChannel        154
CampaignType             0
AdSpend                  0
ClickThroughRate         0
ConversionRate           0
WebsiteVisits            0
PagesPerVisit            0
TimeOnSite              79
SocialShares             0
EmailOpens               0
EmailClicks              0
PreviousPurchases        0
LoyaltyPoints            0
AdvertisingPlatform      0
AdvertisingTool          0
Conversion               0
dtype: int64

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           2010 non-null   int64  
 1   Age                  2010 non-null   int64  
 2   Gender               2010 non-null   object 
 3   Income               1931 non-null   float64
 4   CampaignChannel      1856 non-null   object 
 5   CampaignType         2010 non-null   object 
 6   AdSpend              2010 non-null   object 
 7   ClickThroughRate     2010 non-null   float64
 8   ConversionRate       2010 non-null   float64
 9   WebsiteVisits        2010 non-null   int64  
 10  PagesPerVisit        2010 non-null   float64
 11  TimeOnSite           1931 non-null   float64
 12  SocialShares         2010 non-null   int64  
 13  EmailOpens           2010 non-null   int64  
 14  EmailClicks          2010 non-null   int64  
 15  PreviousPurchases    2010 non-null   i

As we can see all the types seems to be ok but Adspend which should be a numerical value

In [81]:
df.nunique()

CustomerID             2010
Age                      52
Gender                    2
Income                 1923
CampaignChannel           6
CampaignType              4
AdSpend                2010
ClickThroughRate       2010
ConversionRate         2010
WebsiteVisits            50
PagesPerVisit          2010
TimeOnSite             1931
SocialShares            100
EmailOpens               20
EmailClicks              23
PreviousPurchases        10
LoyaltyPoints          1659
AdvertisingPlatform       1
AdvertisingTool           1
Conversion                2
Age_categorized           4
dtype: int64

Advertising Tool and Advertising Platform have only one unique value, therefore don't provide much information for us

<h1>Univariate Analysis & Data Cleanning</h1>

<h3>Campaign Channel

In [68]:
describe_categorical_col(df, 'CampaignChannel')

================== CampaignChannel ==================
          Group  Count      %
0           PPC    422  22.74
1      Referral    404  21.77
2  Social Media    343  18.48
3           SEO    334  18.00
4         Email    247  13.31
5         email    106   5.71


As we can see, Email and email are two separate groups which is wrong and we need to merge them

In [69]:
df['CampaignChannel'] = df['CampaignChannel'].str.replace('email', 'Email')

In [70]:
describe_categorical_col(df, 'CampaignChannel')

================== CampaignChannel ==================
          Group  Count      %
0           PPC    422  22.74
1      Referral    404  21.77
2         Email    353  19.02
3  Social Media    343  18.48
4           SEO    334  18.00


Okay, everything is good so far. let's go for the missing values

In [71]:
df['CampaignChannel'].isna().sum()

154

In [72]:
bins=[18, 30, 45, 60, 70]
bin_labels = ['Young', 'Adult', 'Middle-Aged', 'Old']
categorize_numerical_col('Age', bins=bins, bins_name=bin_labels)

In [73]:
df[df['CampaignChannel'].isna()][['Age_categorized']].value_counts()

Age_categorized
Adult              55
Middle-Aged        49
Old                29
Young              17
Name: count, dtype: int64

In [74]:
for camp in ['Referral', 'PPC', 'Email', 'SEO', 'Social Media']:
    male = df[df['CampaignChannel'] == camp][['Gender']].value_counts()['Male']
    female = df[df['CampaignChannel'] == camp][['Gender']].value_counts()['Female']
    print(f'male to female ratio for {camp}: {male/female}')

male to female ratio for Referral: 0.7046413502109705
male to female ratio for PPC: 0.5687732342007435
male to female ratio for Email: 0.5829596412556054
male to female ratio for SEO: 0.5829383886255924
male to female ratio for Social Media: 0.9055555555555556


In [75]:
male = df[df['CampaignChannel'].isna()][['Gender']].value_counts()['Male']
female = df[df['CampaignChannel'].isna()][['Gender']].value_counts()['Female']
print(f'male to female ratio for {camp}: {male/female}')

male to female ratio for Social Media: 0.49514563106796117


The missing data are not small so if we just replace them with mode, it may conciderbly affect the analysis. Therefore, we create 'Other' to solve this.

In [77]:
df['CampaignChannel'] = df['CampaignChannel'].fillna('Other')

In [78]:
describe_categorical_col(df, 'CampaignChannel')

================== CampaignChannel ==================
          Group  Count      %
0           PPC    422  21.00
1      Referral    404  20.10
2         Email    353  17.56
3  Social Media    343  17.06
4           SEO    334  16.62
5         Other    154   7.66


As we can see PPC or PayPerClick campaign has the most amount of users but not by far. 

--------------

<h3>CampaignType</h3>

In [79]:
describe_categorical_col(df, 'CampaignType')

================== CampaignType ==================
           Group  Count      %
0     Conversion    539  26.82
1  Consideration    502  24.98
2      Retention    490  24.38
3      Awareness    479  23.83


No missing data here, and users are nearly eaqually distributed between campaign types.

------------------------------------------

<h3>AdvertisingPlatform</h3>

In [80]:
describe_categorical_col(df, 'AdvertisingPlatform')

================== AdvertisingPlatform ==================
      Group  Count      %
0  IsConfid   2010  100.0


As we can see this column has only one value so we can neglect it.

-----------------

<h3>AdvertisingTool</h3>

In [83]:
describe_categorical_col(df, 'AdvertisingTool')

================== AdvertisingTool ==================
        Group  Count      %
0  ToolConfid   2010  100.0


Same goes with this column

----------------------